# Aggregation

## **Getting Started**

This week we will be learning how to how to aggregate data using queries.

**Before beginning:** It is recommended that you open/download the _guitar store ERD_ that is available in the Github repo from the Create 3 assignment so you can review it during this assignment.

## **Aggregates**

We can use SQL code to perform aggregate functions on our data. This allows us to do calculations like summing, finding the maximum, or taking an average. These aggregate functions can be added to the select statement of our queries like the code cell below. In this code cell we are going to find the total amount of orders that have been placed.

In [ ]:
SELECT COUNT(*)
FROM gs_orders

In the above query, we did a count operation on all records in the order table. We used the \* because we wanted to count all records in the table, without referencing a specific column. This returned 41 records. The result did not have a name for our column because we were doing an operation. It is good practice to alias columns like this so that the output is clear. Below is an example of how we can use AS to alias a column.

In [ ]:
SELECT COUNT(*) AS num_of_orders
FROM gs_orders

✏️ **Practice**

Try making a new query that will show the sum of all the tax charged in the order table. Name the aggregate field "TotalTax"

In [ ]:
--Type your query below this line
SELECT SUM(tax_amount) AS TotalTax
FROM gs_orders;

If done correctly, your query should return 2541.96

When using aggregates, you are able to show aggregate values as subtotals of different groups. For example, instaed of showing just the total count, we could show the count of orders grouped by any of the other fields in the table. To do this, we use a GROUP BY clause. In the below code cell we will query the total number of orders by CardType.

In [ ]:
SELECT o.card_type, COUNT(*) AS total_orders
FROM gs_orders AS o
GROUP BY o.card_type

You can see that the above query returns the total number of orders for each type of card. If we add the values from the TotalOrders column, we get 41, which is out total number of orders from before. By using joins and aggregations, we can make our queries more powerful. Below is an example that shows how many products are in each category.

In [ ]:
SELECT c.category_name, COUNT(*) AS total_products
FROM gs_categories as c
JOIN gs_products as p ON c.category_id = p.category_id
GROUP BY c.category_name

In the above cell we were able to group records/data from a related table using our JOIN statement.

When working with aggregates, every column of your output must either be some type of grouping, or an aggregate. Simply put, if you aren't using the column for aggregation, it must be part of your GROUP BY clause.

Below is a query where we do not include all columns in either a group by or an aggregate function. It will result in an error.

In [ ]:
SELECT c.category_id, c.category_name, COUNT(*) AS total_products
FROM gs_categories as c
JOIN gs_products as p ON c.category_id = p.category_id
GROUP BY c.category_name

To fix the above query, we need to include _c.category\_id_ in our GROUP BY clause. Once that is done, all columns in the SELECT clause will be included in an aggregate function, or in the GROUP BY clause.

In [ ]:
SELECT c.category_id, c.category_name, COUNT(*) AS total_products
FROM gs_categories as c
JOIN gs_products as p ON c.category_id = p.category_id
GROUP BY c.category_id, c.category_name

We can apply criteria to these queries as well. In the below code cell we will be using SELECT, FROM, WHERE, JOIN, and GROUP BY to see how many products there are in each category that are listed at less than $800.

In [ ]:
SELECT c.category_name, COUNT(*) AS total_products
FROM gs_categories as c
JOIN gs_products AS p ON c.category_id = p.category_id
WHERE p.list_price < 800
GROUP BY c.category_name

The above cell gives a return that tells us that two of the basses, two of the drums, and four of the guitars in the product table have a list price of less than $800.

Notice that we had to put our WHERE clause before our GROUP BY clause. SQL is very picky about the order of your statements. As a general rule, your statements should be in this order SELECT -\> FROM -\> JOIN -\> WHERE -\> GROUP BY -\> HAVING -\> ORDER BY.  Remeber, this is different than our SQL order of operations. SQL statements are not executed in the order that they are written.

This would be an excellent time to cover the HAVING clause. HAVING is similar to WHERE, but applies after aggregation has occured. When using WHERE, you are filtering the data before doing any aggregation, like when we were checking the price just now. When you are using HAVING, you are checking the value after aggregation, so you would be filtering based on the _new_ value for that field. In the below code cell is an example of HAVING that limits the results of our query to Categories that only offer 2 products.

In [ ]:
SELECT c.category_name, COUNT(*) AS total_products
FROM gs_categories AS c
JOIN gs_products AS p ON c.category_id = p.category_id
GROUP BY c.category_name
HAVING COUNT(*) = 2

In the above cell we can see that we are no longer pulling in the guitars category because it does not have exactly two products. In SQL there is an order of operations, just like in math. Below is the full order:

1. FROM (Selecting which tables)
2. WHERE (Filtering the tables)
3. GROUP BY (Grouping data into categories)
4. HAVING (Filtering based on results of aggregation)
5. SELECT (Column selection, aggregation, and column aliasing. Since this comes after having, you can't use column aliases in your having statement)
6. ORDER BY (Sorting)
7. LIMIT (Limiting amount of rows in result)

Up to this point, we have only used the COUNT function. There are other aggregation functions that we can use, such as:

- SUM (Add all column values together
- AVG (Get average of column values)
- MIN (Return lowest column value)
- MAX (Return highest column value)

These functions are meant for numeric data types and must be applied to a specific column, unlike the COUNT function.

  

In the below code cell we will use the SUM function to show the total amount charged for shipping on all orders.

In [ ]:
SELECT SUM(o.ship_amount) AS total_ship_amount
FROM gs_orders as o

We can use a GROUP BY clause with these aggregrate functions as well. In the below cell we will group the total shipping amount by the customer id to see how much each customer is paying in shipping.

In [ ]:
SELECT o.customer_id, SUM(o.ship_amount) AS total_ship_amount
FROM gs_orders as o
GROUP BY o.customer_id

When needed, you can include multiple aggregate functions within your return. In the below example we will add the count of orders to our existing query. This will show the total ship amount for each customer, as well as how many orders they placed.

In [ ]:
SELECT o.customer_id, SUM(o.ship_amount) AS total_ship_amount, COUNT(*) AS total_orders
FROM gs_orders as o
GROUP BY o.customer_id

Just like we can alias columns that are created using aggregate operations, we can alias columns that use mathmatical operations. If we wanted to create a query to show the total amount that a customer paid for an item before tax and shipping, we would need to subtract the discount amount from the item price, and then multiply the result time the quantity. In the below query we create a custom column named subtotal to handle this operation.

In [ ]:
SELECT oi.item_id, oi.item_price, oi.discount_amount, oi.quantity, (oi.item_price - oi.discount_amount) * oi.quantity AS subtotal
FROM gs_order_items as oi

In the above cell we added an operation to our SELECT clause so that we could include a calculated column in our output. We started by putting the difference of item price and discount amount in parethesis to force that operation to be executed first, after that we multiply the result by quantity and alias the final output as subtotal. Now we can easily see the price of an order before taxes and shipping. While it was not necessary to return the columns used in our operation in our final output, I went ahead and included them so that you could check the math and see how the query is working.

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Now that you have some experience with custom columns, let's apply this same logic to our aggregate query we were working on before. When we left off we had the sum of shipping charges as well as the number of orders for each customer. Let's add a custom column that shows the amount of shipping per order for each customer.&nbsp;</span>

In [ ]:
--While it is not necessary to put each column on it's own line, this particular example is easier to read if we do so.
SELECT o.customer_id,
SUM(o.ship_amount) AS total_ship_amount,
COUNT(*) AS total_orders,
SUM(o.ship_amount) / COUNT(*) AS ship_amount_per_order
FROM gs_orders as o
GROUP BY o.customer_id

Note in the above example that we divided the aggregate values by each other. We are able to do this because in the SQL order of operations, aggregation is completed during the GROUP BY operations, so the data has been aggregated and can be operated on further by the time the compiler executes SELECT operations.

  

It is also for this reason that we cannot use the aggregate column aliases in our custom columns. Those aliases are assigned at the end of the order of operations during the SELECT phase, so they do not exists during the GROUP BY phase when we are aggregating. This means that we cannot write custom aggregation columns like _total\_ship\_amount / total\_orders AS ship\_amount\_per\_order_.

## **Exercises**

If you have not already, I would strongly recommend having access to the ERD linked in Github for these exercises

1\. Using the below code cell, return the amount of orders that each customer has placed. The query should include the first name, last name, and total\_orders as a customer column.

**First 5 rows of correct solution**
|first_name|last_name|total_orders|
|---|---|---|
|Cammy|Albares|1|
|Minna|Amigon|1|
|Christine|Brown|1|
|James|Butt|1|
|Kiley|Caldarera|1|

In [ ]:
--Insert your code below this line.
SELECT c.first_name, c.last_name, COUNT(o.order_id) AS total_orders
FROM gs_customers AS c
JOIN gs_orders AS o ON c.customer_id = o.customer_id
GROUP BY c.first_name, c.last_name
ORDER BY c.first_name ASC;

2\. Using the below code cell, return only customers that have placed more than 1 order. Only return the first name, last name, and email address fields.

**First 5 rows of correct solution**
|first_name|last_name|email_address|
|---|---|---|
|Allan|Sherwood|allan.sherwood@yahoo.com|
|Bette|Nicka|bette_nicka@cox.net|
|David|Goldstein|david.goldstein@hotmail.com|
|Heather|Esway|heatheresway@mac.com|
|Kris|Marrier|kris@gmail.com|

In [ ]:
--Insert your code below this line.
SELECT c.first_name, c.last_name, c.email_address
FROM gs_customers AS c
JOIN gs_orders o ON c.customer_id = o.customer_id
GROUP BY c.first_name, c.last_name, c.email_address
HAVING COUNT(o.order_id) > 1;

3\. Using the below code cell, return the total quantity of each product that has been ordered by all customers. Include two columns in your output: product name and total ordered.

**First 5 rows of correct solution**
|product_name|total_ordered|
|---|---|
|Fender Precision|2|
|Fender Stratocaster|7|
|Gibson Les Paul|12|
|Gibson SG|9|
|Hofner Icon|3|

In [ ]:
--Insert your code below this line.
SELECT p.product_name, SUM(o.quantity) AS total_ordered
FROM gs_products AS p
JOIN gs_order_items AS o ON p.product_id = o.product_id
GROUP BY p.product_name
ORDER BY p.product_name ASC;

4\. Using the below code cell, return a report of how many customers are from each state in the dataset. Include two columns in your output: state and total customers.

Join gs\_addresses to gs\_customer using address\_id and shipping\_address\_id

**First 5 rows of correct solution**
|state_code|total_customers|
|---|---|
|AK|5|
|AR|1|
|AZ|8|
|CA|70|
|CO|9|


In [ ]:
--Insert your code below this line.
SELECT a.state_code, COUNT(c.customer_id) AS total_customers
FROM gs_customers c
JOIN gs_addresses a ON c.shipping_address_id = a.address_id
GROUP BY a.state_code
ORDER BY a.state_code ASC;

## **Scenario**

The owner of Guitar Store wants you to create a query that will show the total amount charged for each order before shipping & tax, the order id, as well as information about the customer who placed the order. Your query will need to get the sum of all subtotals with in order and pull in the name and email of the customer.

For complex queries like this, do not try to write them from top to bottom. Focus on one part of the problem and get that query working, then build and add on to it until you have the intended result.

Create your query in the code cell below:

**First 5 rows of correct solution**
|order_id|first_name|last_name|email_address|order_total|
|---|---|---|---|---|
|1|Allan|Sherwood|allan.sherwood@yahoo.com|839.30|
|2|Barry|Zimmer|barryz@gmail.com|303.79|
|3|Allan|Sherwood|allan.sherwood@yahoo.com|1461.31|
|4|Christine|Brown|christineb@solarone.com|1678.60|
|5|David|Goldstein|david.goldstein@hotmail.com|299.00|


In [ ]:
--Insert your code below this line
SELECT o.order_id, c.first_name, c.last_name, c.email_address, 
SUM(oi.quantity * (oi.item_price - oi.discount_amount)) AS order_total
FROM gs_orders AS o
JOIN gs_customers AS c ON o.customer_id = c.customer_id
JOIN gs_order_items AS oi ON o.order_id = oi.order_id
GROUP BY o.order_id, c.first_name, c.last_name, c.email_address
ORDER BY o.order_id ASC;